In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

### Tables in the process

In [2]:
cols = 'trade name qty_x price reason_x qty_y buy_target sell_target reason_y'.split()
colt = 'trade name qty price active reason market'.split()

In [3]:
format_dict = {
    'qty':'{:,}','qty_x':'{:,}','qty_y':'{:,}','available_qty':'{:,}','eps_a':'{:.2f}','eps_b':'{:.2f}',
    'volbuy':'{:,}','volsell':'{:,}','volbal':'{:,}',
    'pe':'{:.2f}','pbv':'{:.2f}','max':'{:.2f}','min':'{:.2f}','dividend':'{:.4f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}','daily_volume':'{:,.2f}','dly_vol':'{:,.2f}',
    'price':'{:.2f}','target_price':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}','amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}','percent':'{:,.2f}',
    'q_amt':'{:,}','y_amt':'{:,}','aq_amt':'{:,}','ay_amt':'{:,}','yoy_gain':'{:,}','acc_gain':'{:,}',    
    'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
    'yoy_pct':'{:.2f}%','acc_pct':'{:.2f}%','yield':'{:.2f}%',
    'q_amt': '{:,}','y_amt': '{:,}','yoy_gain': '{:,}','q_amt_c': '{:,}','q_amt_p': '{:,}','aq_amt': '{:,}','ay_amt': '{:,}',
    'acc_gain': '{:,}','latest_amt': '{:,}','previous_amt': '{:,}','inc_amt': '{:,}',
    'inc_amt_pq': '{:,}','inc_amt_py': '{:,}',    'latest_amt_q': '{:,}','previous_amt_q': '{:,}','inc_amt_q': '{:,}',
    'latest_amt_y': '{:,}','previous_amt_y': '{:,}','inc_amt_y': '{:,}','kind_x': '{:,}',
    'inc_pct': '{:.2f}%','inc_pct_q': '{:.2f}%','pct': '{:.2f}%','percent': '{:.2f}%',
    'inc_pct_y': '{:.2f}%','inc_pct_pq': '{:.2f}%','inc_pct_py': '{:.2f}%','mean_pct': '{:.2f}%','std_pct': '{:.2f}%',      
}

### Begin of Loop

In [4]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.sort_values(['trade','name'],ascending=[True,True]).style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,AEONTS,"1,200",180.00,"216,000.00",1L,SET100,1
1,B,BGC,"10,000",10.00,"100,000.00",DOS,SET,1
2,B,NOBLE,"21,000",5.85,"122,850.00",RD15%,SET,0
3,B,PTT,"3,000",36.50,"109,500.00",RD05%,SET50,1
4,B,PTTGC,"1,200",54.25,"65,100.00",RD10pct,SET50,1
5,B,RATCH,"3,000",42.75,"128,250.00",RD05%,SET50,1
6,B,RCL,"1,500",41.50,"62,250.00",RD05pct,SET100,1
7,B,ROJNA,"18,000",6.30,"113,400.00",6L,SET,1
8,B,SCC,300,374.00,"112,200.00",RD05pct,SET50,2
9,B,SMT,"18,000",5.00,"90,000.00",None,SET,0


In [5]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlite)
stocks.shape

(54, 18)

In [6]:
df_merge = pd.merge(orders,stocks,on='name',how='inner')
df_merge.shape

(27, 25)

In [7]:
qty_mask = (df_merge.qty_x != df_merge.qty_y)
df_merge[qty_mask][cols].style.format(format_dict)

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y
15,B,TSTH,"45,000",1.42,RD05pct,"75,000",1.42,1.52,CP1S
25,S,SIS,"1,500",46.50,15pct,"3,000",0.00,46.50,15pct


In [8]:
buy_price_mask = (df_merge.price != df_merge.buy_target) & (df_merge.trade == 'B')
df_merge[buy_price_mask][cols].style.format(format_dict)

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y


In [9]:
sell_price_mask = (df_merge.price != df_merge.sell_target) & (df_merge.trade == 'S')
df_merge[sell_price_mask][cols].style.format(format_dict)

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y
21,S,IVL,"3,000",49.00,20pct,"3,000",0.00,46.25,10pct


In [10]:
reason_mask = (df_merge.reason_x != df_merge.reason_y)
df_merge[reason_mask][cols].style.format(format_dict)

,trade,name,qty_x,price,reason_x,qty_y,buy_target,sell_target,reason_y
0,B,AEONTS,"1,200",180.00,1L,"1,200",180.00,0.00,3L
9,B,SMT,"18,000",5.00,None,"18,000",5.00,0.00,6L
13,B,TOP,"3,000",51.00,RU05pct,"3,000",51.00,55.00,ROUND
15,B,TSTH,"45,000",1.42,RD05pct,"75,000",1.42,1.52,CP1S
21,S,IVL,"3,000",49.00,20pct,"3,000",0.00,46.25,10pct


### End of Loop process

In [11]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

orders[colt].to_csv(output_file, header=True, index=False)
orders[colt].to_csv(data_file,   header=True, index=False)
orders[colt].to_csv(box_file,    header=True, index=False)

### After call ord-log

In [12]:
def categorise(row):  
    if row['col0'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [13]:
df = pd.read_csv('../data/orders-log.csv', header=None, prefix = 'col',usecols=[0,1,2,3,4,5,6,7,8,9])
df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market','col5': 'qty', 'col6': 'target','col7': 'current', 'col8': 'chg', 'col9': 'percent'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df.shape

(28, 11)

In [14]:
from itables import show
show(df[['trans','name','spd','reason','qty','target','current','chg','percent']])

<IPython.core.display.Javascript object>

trans,name,spd,reason,qty,target,current,chg,percent


In [15]:
sql = '''
UPDATE stocks
SET buy_target = 0
WHERE buy_target IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [16]:
sql = '''
UPDATE stocks
SET sell_target = 0
WHERE sell_target IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [17]:
sql = '''
UPDATE stocks
SET available_qty = 0
WHERE available_qty IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [18]:
sql = '''
UPDATE stocks
SET cost = 0
WHERE cost IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [19]:
sql = '''
UPDATE stocks
SET qty = 0
WHERE qty IS NULL'''
rp = conlite.execute(sql)
rp.rowcount

0

In [20]:
qty_mask = (stocks.qty == 0) & (stocks.status.isin(['B','I','O','S']))
stocks[qty_mask].style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market


In [21]:
buy_mask = (stocks.buy_target == 0) & (stocks.status.isin(['B','O']))
stocks[buy_mask].style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market


In [22]:
sell_mask = (stocks.sell_target == 0) & (stocks.status.isin(['I','S']))
stocks[sell_mask].style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
